In [3]:
%matplotlib inline
#%matplotlib notebook
#%matplotlib widget
import matplotlib 
import numpy as np
import pandas as pd
import os, sys
#import ipywidgets
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection    
 
# use LaTeX, choose nice some looking fonts and tweak some settings
matplotlib.rc('font', family='serif')
matplotlib.rc('font', size=16)
matplotlib.rc('legend', fontsize=16)
matplotlib.rc('legend', numpoints=1)
matplotlib.rc('legend', handlelength=1.5)
matplotlib.rc('legend', frameon=False)
matplotlib.rc('xtick.major', pad=7)
matplotlib.rc('xtick.minor', pad=7)
matplotlib.rc('text', usetex=True)
# matplotlib.rc('text.latex', 
#               preamble=[r'\usepackage[T1]{fontenc}',
#                         r'\usepackage{amsmath}',
#                         r'\usepackage{txfonts}',
#                         r'\usepackage{textcomp}'])

matplotlib.rc('figure', figsize=(12, 9))

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device('cuda:0')

V. 소프트맥스 회귀
===

이번 챕터에서는 3개 이상의 선택지로부터 1개를 선택하는 문제인 다중 클래스 분류(Multi-Class classification)를 풀기 위한 소프트맥스 회귀에 대해서 학습합니다.

## 4.1 원 핫 인코딩 (One-hot encoding)
**원-핫 인코딩**은 선택해야 하는 선택지의 개수만큼의 차원을 가지면서, 각 선택지의 인덱스에 해당하는 원소에는 1, 나머지 원소는 0의 값을 가지도록 하는 표현 방법. 원-핫 인코딩으로 표현된 벡터를 **원-핫 벡터**(one-hot vector)라 한다. 

## 4.2 소프트맥스 회귀(Softmax Regression) 이해하기
앞서 로지스틱 회귀를 통해 2개의 선택지 중에서 1개를 고르는 이진 분류(Binary Classification)를 풀어봤습니다. 이번 챕터에서는 소프트맥스 회귀를 통해 3개 이상의 선택지 중에서 1개를 고르는 다중 클래스 분류(Multi-Class Classification)를 실습해봅시다.

#### 소프트맥스 함수
$N$ 차원 벡터에서 $i$번째 원소가 $z_i$ 이며 $i$번째 클라스가 정답인 확률이 $p_i$ 일 때, 소프트맥스 함수는 $p_i$ 를 다음과 같이 정의한다.
$$
p_i = \dfrac{e^{z_i}}{\displaystyle\sum_{j=1}^N e^{z_i}}
$$

#### 소프트맥스 비용 함수

$N$ 개의 클라스로 분류하는 소프트맥스 회귀에서는 비용함수로 다음과 같이 정의된 크로스엔트로피 함수를 사용한다. $y_j$ 를 실제값 원-핫 벡터의 $j$ 번째 인덱스이고, $p_j$ 가 $j$ 번째 클래스일 확률 일 때,
$$
\text{cost}(W) = -\dfrac{1}{N}\sum_{j=1}^N y_j \log p_j
$$
이다.$N=2$ 일 때 위의 비용함수는 Logistic 회귀에서의 비용함수와 같음을 쉽게 알 수 있다.

In [17]:
def softmax1(vervose=False):
    torch.manual_seed(1)
    # 입력값
    z = torch.rand(3, 5, requires_grad=True)


    hypothesis = F.softmax(z, dim=0)
    if vervose :
        print("hypothesis=", hypothesis)
    y = torch.randint(5, (3,)).long()
    if vervose :
        print("y = ", y)
    # 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
    y_one_hot = torch.zeros_like(hypothesis) 
    y_one_hot.scatter_(1, y.unsqueeze(1), 1)
#     print(y.unsqueeze(1))
#     print(y_one_hot)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    print("cost = ", cost)
softmax1(True)

hypothesis= tensor([[0.3412, 0.2938, 0.2671, 0.4002, 0.2469],
        [0.3559, 0.3306, 0.3796, 0.3393, 0.3719],
        [0.3029, 0.3756, 0.3533, 0.2605, 0.3812]], grad_fn=<SoftmaxBackward>)
y =  tensor([0, 2, 1])
cost =  tensor(1.0078, grad_fn=<MeanBackward0>)


In [19]:
USE_CUDA = torch.cuda.is_available() 

In [20]:
USE_CUDA

True

In [30]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random, time

def mnist(pu=0):
    t1=time.time()
    ddv = ('cpu', 'cuda:0', 'cuda:1')
    device = torch.device(ddv[pu])
    if pu in (1, 2):
        torch.cuda.manual_seed_all(777)
    else :
         torch.manual_seed(1)
        
    # hyperparameters
    training_epochs = 15
    batch_size = 100

    mnist_train = dsets.MNIST(root='MNIST_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

    mnist_test = dsets.MNIST(root='MNIST_data/',
                             train=False,
                             transform=transforms.ToTensor(),
                             download=True)

    data_loader = DataLoader(dataset=mnist_train,
                                              batch_size=batch_size, # 배치 크기는 100
                                              shuffle=True,
                                              drop_last=True)
    
    # MNIST data image of shape 28 * 28 = 784
    linear = nn.Linear(784, 10, bias=True).to(device)
    
    
    # 비용 함수와 옵티마이저 정의
    criterion = nn.CrossEntropyLoss().to(device) # 내부적으로 소프트맥스 함수를 포함하고 있음.
    optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

    for epoch in range(training_epochs): # 앞서 training_epochs의 값은 15로 지정함.
        avg_cost = 0
        total_batch = len(data_loader)

        for X, Y in data_loader:
            # 배치 크기가 100이므로 아래의 연산에서 X는 (100, 784)의 텐서가 된다.
            X = X.view(-1, 28 * 28).to(device)
            # 레이블은 원-핫 인코딩이 된 상태가 아니라 0 ~ 9의 정수.
            Y = Y.to(device)

            optimizer.zero_grad()
            hypothesis = linear(X)
            cost = criterion(hypothesis, Y)
            cost.backward()
            optimizer.step()

            avg_cost += cost / total_batch

        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    t2=time.time()
    print('Learning finished for '+str(t2-t1)+" sec")

    # 테스트 데이터를 사용하여 모델을 테스트한다.
    with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
        X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
        Y_test = mnist_test.test_labels.to(device)

        prediction = linear(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

        # MNIST 테스트 데이터에서 무작위로 하나를 뽑아서 예측을 해본다
        r = random.randint(0, len(mnist_test) - 1)
        X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
        Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

        print('Label: ', Y_single_data.item())
        single_prediction = linear(X_single_data)
        print('Prediction: ', torch.argmax(single_prediction, 1).item())

        plt.imshow(mnist_test.test_data[r:r + 1].view(28, 28), cmap='Greys', interpolation='nearest')
        plt.show()
    print('testing finished for '+str(time.time()-t2)+" sec")

In [33]:
mnist(1)

0it [00:00, ?it/s]

HTTPError: HTTP Error 503: Service Unavailable